In [6]:
def calc_pr(players):
    pr = {}
    for player, (elo, score, games) in players.items():
        avg_opp = sum(e for p, (e, s, g) in players.items() if p != player)
        pr[player] = (avg_opp + 400 * score) / games
    return pr

In [7]:
calc_pr({'a': (1000, 0, 2), 'b': (1000, -1, 2), 'c': (1000, 1, 2)})

{'a': 1000.0, 'b': 800.0, 'c': 1200.0}

In [18]:
def calc_elo(players, num_iter=10):
    '''
    Calculates relative elo such that all players have a elo equal to their performance rating

    players: dict of key: <player_name>, value: tuple (<score>, <games>) where score is (#wins - #losses) and games is number of games played
    i.e. a player that wins all games has score == games and a player that loses all games has score == -games
    requries that no player have won all or lost all games for convergence (otherwise elo = +- infinity)
    '''
    d = {player : (1000, score, games) for player, (score, games) in players.items()}
    for i in range(num_iter):
        pr = calc_pr(d)
        d = {player: (pr[player], score, games) for player, (elo, score, games) in d.items()}
    return d

In [20]:
calc_elo({'a': (0, 2), 'b': (-1, 2), 'c': (1, 2)}, 10)

{'a': (1000.0, 0, 2), 'b': (866.796875, -1, 2), 'c': (1133.203125, 1, 2)}